## Development of resource use functions

In [47]:
import pandas as pd
import _utilisation_result_calculation
import _processing_functions

run_results = pd.read_csv("../data/run_results.csv")
run_results = _processing_functions.make_callsign_column(run_results)

run_results.head()


,P_ID,run_number,time_type,event_type,timestamp,timestamp_dt,day,hour,weekday,month,qtr,callsign_group,vehicle_type,ampds_card,age,sex,hems_result,outcome,callsign
0,1,1,70,resource_preferred_resource_group,0.0,2025-02-14 08:00:00,Fri,8,weekday,2,1,NaN,NaN,12,60.0,Female,NaN,NaN,NaN
1,1,1,helicopter,resource_preferred_vehicle_type,0.0,2025-02-14 08:00:00,Fri,8,weekday,2,1,NaN,NaN,12,60.0,Female,NaN,NaN,NaN
2,1,1,Preferred resource not and available but other...,resource_preferred_outcome,0.0,2025-02-14 08:00:00,Fri,8,weekday,2,1,70.0,car,12,60.0,Female,NaN,NaN,C70
3,1,1,CC70,resource_use,0.0,2025-02-14 08:00:00,Fri,8,weekday,2,1,70.0,car,12,60.0,Female,NaN,NaN,C70
4,1,1,arrival,arrival_departure,0.0,2025-02-14 08:00:00,Fri,8,weekday,2,1,70.0,car,12,60.0,Female,NaN,NaN,C70


In [48]:
params = pd.read_csv("../data/run_params_used.csv")
params

,parameter,value
0,sim_duration,525600.0
1,warm_up_duration,0.0
2,sim_start_date,2025-02-14 08:00:00
3,sim_end_date,2026-02-14 08:00:00
4,warm_up_end_date,2025-02-14 08:00:00
5,amb_data,False
6,model_exec_time,2025-02-14 08:42:29.863301
7,summer_start_date,2025-04-01
8,winter_start_date,2025-10-01


In [49]:
params[params["parameter"] == "sim_duration"]['value'].values[0]

'525600.0'

In [50]:
resource_use_only = run_results[run_results["event_type"].isin(["resource_use", "resource_use_end"])]
resource_use_only

,P_ID,run_number,time_type,event_type,timestamp,timestamp_dt,day,hour,weekday,month,qtr,callsign_group,vehicle_type,ampds_card,age,sex,hems_result,outcome,callsign
3,1,1,CC70,resource_use,0.0,2025-02-14 08:00:00,Fri,8,weekday,2,1,70.0,car,12,60.0,Female,NaN,NaN,C70
8,2,1,CC71,resource_use,60.0,2025-02-14 09:00:00,Fri,9,weekday,2,1,71.0,car,10,63.0,Male,NaN,NaN,C71
12,1,1,CC70,resource_use_end,65.0,2025-02-14 09:05:00,Fri,8,weekday,2,1,70.0,car,12,60.0,Female,Patient Treated (not conveyed),Conveyed by land without DAA,C70
20,3,1,H71,resource_use,120.0,2025-02-14 10:00:00,Fri,10,weekday,2,1,71.0,helicopter,27,24.0,Female,NaN,NaN,H71
25,2,1,CC71,resource_use_end,148.0,2025-02-14 10:28:00,Fri,9,weekday,2,1,71.0,car,10,63.0,Male,Patient Conveyed,Conveyed by land with DAA,C71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101794,1627,5,CC72,resource_use_end,524668.0,2026-02-13 16:28:00,Fri,15,weekday,2,1,72.0,car,29,60.0,Male,Patient Treated (not conveyed),Conveyed by land without DAA,C72
101800,1628,5,H70,resource_use,524760.0,2026-02-13 18:00:00,Fri,18,weekday,2,1,70.0,helicopter,29,12.0,Male,NaN,NaN,H70
101806,1628,5,H70,resource_use_end,524786.0,2026-02-13 18:26:00,Fri,18,weekday,2,1,70.0,helicopter,29,12.0,Male,Stand Down En Route,Unknown,H70
101812,1629,5,H70,resource_use,525060.0,2026-02-13 23:00:00,Fri,23,weekday,2,1,70.0,helicopter,23,67.0,Female,NaN,NaN,H70


In [51]:
resource_use_wide = resource_use_only[["P_ID", "run_number", "event_type", "timestamp_dt", "callsign_group", "vehicle_type", "callsign"]].pivot(index=["P_ID", "run_number", "callsign_group", "vehicle_type", "callsign"], columns="event_type", values="timestamp_dt").reset_index()

resource_use_wide

event_type,P_ID,run_number,callsign_group,vehicle_type,callsign,resource_use,resource_use_end
0,1,1,70.0,car,C70,2025-02-14 08:00:00,2025-02-14 09:05:00
1,1,2,70.0,car,C70,2025-02-14 11:00:00,2025-02-14 11:43:00
2,1,3,71.0,car,C71,2025-02-14 13:00:00,2025-02-14 14:34:00
3,1,4,70.0,car,C70,2025-02-14 08:00:00,2025-02-14 08:23:00
4,1,5,71.0,car,C71,2025-02-14 09:00:00,2025-02-14 09:30:00
...,...,...,...,...,...,...,...
7319,1679,4,71.0,car,C71,2026-02-12 09:00:00,2026-02-12 12:41:00
7320,1680,4,70.0,helicopter,H70,2026-02-13 08:00:00,2026-02-13 08:57:00
7321,1681,4,70.0,helicopter,H70,2026-02-13 10:00:00,2026-02-13 10:53:00
7322,1682,4,70.0,helicopter,H70,2026-02-13 17:00:00,2026-02-13 17:54:00


In [52]:
resource_use_wide['resource_use'].isna().sum()

0

In [53]:
resource_use_wide['resource_use_end'].isna().sum()

1

In [54]:
type(resource_use_wide['resource_use_end'][0])

str

In [55]:
import _vehicle_calculation

daily_availability, total_avail_hours, total_avail_minutes = _vehicle_calculation.calculate_available_hours(
        params, rota_path="../data/hems_rota_used.csv"
        )

daily_availability

c:\DAAT\DAA_DES\app\_vehicle_calculation.py:78: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\DAAT\DAA_DES\app\_vehicle_calculation.py:85: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,is_summer,H70,CC70,H71,CC71,CC72
0,2025-02-14,False,11,11,2,2,2
1,2025-02-15,False,19,19,10,10,10
2,2025-02-16,False,19,19,10,10,10
3,2025-02-17,False,19,19,10,10,10
4,2025-02-18,False,19,19,10,10,10
...,...,...,...,...,...,...,...
361,2026-02-10,False,19,19,10,10,10
362,2026-02-11,False,19,19,10,10,10
363,2026-02-12,False,19,19,10,10,10
364,2026-02-13,False,19,19,10,10,10


In [56]:
total_avail_hours

,callsign,total_available_hours_in_sim,callsign_group
0,H70,6935,70
1,C70,6935,70
2,H71,4016,71
3,C71,3650,71
4,C72,3650,72


In [57]:
total_avail_minutes

,callsign,total_available_minutes_in_sim,callsign_group
0,H70,416100,70
1,C70,416100,70
2,H71,240960,71
3,C71,219000,71
4,C72,219000,72


In [58]:
resource_use_wide["resource_use_duration"] = _processing_functions.calculate_time_difference(
        resource_use_wide, 'resource_use', 'resource_use_end', unit='minutes'
        )

utilisation_df_per_run = (
        resource_use_wide.groupby(['run_number', 'vehicle_type', 'callsign'])
        [["resource_use_duration"]]
        .sum()
        )

utilisation_df_per_run


event_type                        resource_use_duration
run_number vehicle_type callsign                       
1          car          C70                     73246.0
                        C71                     22941.0
                        C72                     23569.0
           helicopter   H70                     40950.0
                        H71                      9439.0
2          car          C70                     58411.0
                        C71                     14477.0
                        C72                     20813.0
           helicopter   H70                     28842.0
                        H71                     10375.0
3          car          C70                     41906.0
                        C71                     17418.0
                        C72                     24781.0
           helicopter   H70                     45831.0
                        H71                      9962.0
4          car          C70                     69246.0
                        C71                     28350.0
                        C72                     25570.0
           helicopter   H70                     32356.0
                        H71                     16765.0
5          car          C70                     65124.0
                        C71                     15768.0
                        C72                     28307.0
           helicopter   H70                     35925.0
                        H71                      8987.0

In [59]:
utilisation_df_per_run.reset_index(drop=False).merge(total_avail_minutes, on="callsign")

,run_number,vehicle_type,callsign,resource_use_duration,total_available_minutes_in_sim,callsign_group
0,1,car,C70,73246.0,416100,70
1,1,car,C71,22941.0,219000,71
2,1,car,C72,23569.0,219000,72
3,1,helicopter,H70,40950.0,416100,70
4,1,helicopter,H71,9439.0,240960,71
5,2,car,C70,58411.0,416100,70
6,2,car,C71,14477.0,219000,71
7,2,car,C72,20813.0,219000,72
8,2,helicopter,H70,28842.0,416100,70
9,2,helicopter,H71,10375.0,240960,71


In [60]:
utilisation_df_per_run_by_csg = (
        resource_use_wide.groupby(['callsign_group'])
        [["resource_use_duration"]]
        .sum()
        )
utilisation_df_per_run_by_csg = utilisation_df_per_run_by_csg.reset_index()
utilisation_df_per_run_by_csg



# import re

# total_avail_hours["callsign_group"] = total_avail_hours["callsign"].apply(lambda x: re.sub('\D', '', x))
# total_avail_hours

event_type,callsign_group,resource_use_duration
0,70.0,491837.0
1,71.0,154482.0
2,72.0,123040.0


In [61]:
total_avail_hours_per_csg = total_avail_minutes.groupby('callsign_group').head(1).drop(columns='callsign')
total_avail_hours_per_csg['callsign_group'] =  total_avail_hours_per_csg['callsign_group'].astype('float')
total_avail_hours_per_csg

,total_available_minutes_in_sim,callsign_group
0,416100,70.0
2,240960,71.0
4,219000,72.0


In [62]:
utilisation_df_per_run_by_csg = utilisation_df_per_run_by_csg.merge(total_avail_hours_per_csg, on="callsign_group")
utilisation_df_per_run_by_csg

,callsign_group,resource_use_duration,total_available_minutes_in_sim
0,70.0,491837.0,416100
1,71.0,154482.0,240960
2,72.0,123040.0,219000


In [63]:
n_runs=5
utilisation_df_overall = (
        utilisation_df_per_run.groupby(['callsign', 'vehicle_type'])
        [["resource_use_duration"]]
        .sum()
        )

utilisation_df_overall["resource_use_duration"] = (
    utilisation_df_overall["resource_use_duration"] /
    n_runs
    )

utilisation_df_overall.reset_index()

event_type,callsign,vehicle_type,resource_use_duration
0,C70,car,61586.6
1,C71,car,19790.8
2,C72,car,24608.0
3,H70,helicopter,36780.8
4,H71,helicopter,11105.6


In [64]:
n_runs=5

In [65]:
total_avail_minutes['total_available_minutes_in_all_runs'] = total_avail_minutes['total_available_minutes_in_sim'] * n_runs

In [66]:
total_avail_minutes

,callsign,total_available_minutes_in_sim,callsign_group,total_available_minutes_in_all_runs
0,H70,416100,70,2080500
1,C70,416100,70,2080500
2,H71,240960,71,1204800
3,C71,219000,71,1095000
4,C72,219000,72,1095000


In [67]:
resource_use_wide, utilisation_df_overall, utilisation_df_per_run, utilisation_df_per_run_by_csg = _utilisation_result_calculation.make_utilisation_model_dataframe()
resource_use_wide

c:\DAAT\DAA_DES\app\_vehicle_calculation.py:78: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\DAAT\DAA_DES\app\_vehicle_calculation.py:85: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



event_type,P_ID,run_number,callsign_group,vehicle_type,callsign,resource_use,resource_use_end,resource_use_duration
0,1,1,70.0,car,C70,2025-02-14 08:00:00,2025-02-14 09:05:00,65.0
1,1,2,70.0,car,C70,2025-02-14 11:00:00,2025-02-14 11:43:00,43.0
2,1,3,71.0,car,C71,2025-02-14 13:00:00,2025-02-14 14:34:00,94.0
3,1,4,70.0,car,C70,2025-02-14 08:00:00,2025-02-14 08:23:00,23.0
4,1,5,71.0,car,C71,2025-02-14 09:00:00,2025-02-14 09:30:00,30.0
...,...,...,...,...,...,...,...,...
7319,1679,4,71.0,car,C71,2026-02-12 09:00:00,2026-02-12 12:41:00,221.0
7320,1680,4,70.0,helicopter,H70,2026-02-13 08:00:00,2026-02-13 08:57:00,57.0
7321,1681,4,70.0,helicopter,H70,2026-02-13 10:00:00,2026-02-13 10:53:00,53.0
7322,1682,4,70.0,helicopter,H70,2026-02-13 17:00:00,2026-02-13 17:54:00,54.0


In [68]:
utilisation_df_per_run_by_csg

,callsign_group,resource_use_duration,total_available_minutes_in_sim,perc_time_in_use,PRINT_perc
0,70.0,98847.4,416100,0.237557,23.8%
1,71.0,30896.4,240960,0.128222,12.8%
2,72.0,24608.0,219000,0.112365,11.2%


In [69]:
utilisation_df_overall

,callsign,vehicle_type,resource_use_duration,total_available_minutes_in_sim,callsign_group,perc_time_in_use,PRINT_perc
0,C70,car,62066.6,416100,70,0.149163,14.9%
1,C71,car,19790.8,219000,71,0.090369,9.0%
2,C72,car,24608.0,219000,72,0.112365,11.2%
3,H70,helicopter,36780.8,416100,70,0.088394,8.8%
4,H71,helicopter,11105.6,240960,71,0.046089,4.6%


In [70]:
utilisation_df_per_run

,run_number,vehicle_type,callsign,resource_use_duration,total_available_minutes_in_sim,callsign_group,perc_time_in_use,PRINT_perc
0,1,car,C70,75646.0,416100,70,0.181798,18.2%
1,1,car,C71,22941.0,219000,71,0.104753,10.5%
2,1,car,C72,23569.0,219000,72,0.107621,10.8%
3,1,helicopter,H70,40950.0,416100,70,0.098414,9.8%
4,1,helicopter,H71,9439.0,240960,71,0.039172,3.9%
5,2,car,C70,58411.0,416100,70,0.140377,14.0%
6,2,car,C71,14477.0,219000,71,0.066105,6.6%
7,2,car,C72,20813.0,219000,72,0.095037,9.5%
8,2,helicopter,H70,28842.0,416100,70,0.069315,6.9%
9,2,helicopter,H71,10375.0,240960,71,0.043057,4.3%


In [71]:
import plotly.express as px

px.box(utilisation_df_per_run.reset_index(),
       x="perc_time_in_use",
       y="callsign",
       color="vehicle_type")

In [72]:
px.bar(utilisation_df_overall.reset_index(),
       y="perc_time_in_use",
       x="callsign",
       color="vehicle_type")

In [73]:
_utilisation_result_calculation.make_SIMULATION_utilisation_summary_plot(utilisation_df_overall=utilisation_df_overall)

In [74]:
_utilisation_result_calculation.make_SIMULATION_utilisation_variation_plot(utilisation_df_per_run=utilisation_df_per_run)

In [75]:
utilisation_df_per_run_by_csg

,callsign_group,resource_use_duration,total_available_minutes_in_sim,perc_time_in_use,PRINT_perc
0,70.0,98847.4,416100,0.237557,23.8%
1,71.0,30896.4,240960,0.128222,12.8%
2,72.0,24608.0,219000,0.112365,11.2%


## Available time calculation

In [76]:
import pandas as pd
hems_rota = pd.read_csv("../data/hems_rota_used.csv")
hems_rota[['callsign', 'summer_start', 'winter_start', 'summer_end', 'winter_end']]

,callsign,summer_start,winter_start,summer_end,winter_end
0,H70,7,7,2,2
1,CC70,7,7,2,2
2,H71,7,7,19,17
3,CC71,9,7,19,17
4,CC72,8,8,18,18


In [77]:
params = pd.read_csv("../data/run_params_used.csv")
params

,parameter,value
0,sim_duration,525600.0
1,warm_up_duration,0.0
2,sim_start_date,2025-02-14 08:00:00
3,sim_end_date,2026-02-14 08:00:00
4,warm_up_end_date,2025-02-14 08:00:00
5,amb_data,False
6,model_exec_time,2025-02-14 08:42:29.863301
7,summer_start_date,2025-04-01
8,winter_start_date,2025-10-01


In [78]:
import _processing_functions
from datetime import datetime

In [79]:
warm_up_end = _processing_functions.get_param("warm_up_end_date", params)
warm_up_end = datetime.strptime(warm_up_end, "%Y-%m-%d %H:%M:%S")
sim_end = _processing_functions.get_param("sim_end_date", params)
sim_end = datetime.strptime(sim_end, "%Y-%m-%d %H:%M:%S")


date_range = pd.date_range(start=warm_up_end.date(),
                           end=sim_end.date(),
                           freq='D')
df = pd.DataFrame({'date': date_range})

summer_start = datetime.strptime(_processing_functions.get_param("summer_start_date", params), "%Y-%m-%d")
winter_start = datetime.strptime(_processing_functions.get_param("winter_start_date", params), "%Y-%m-%d")

summer_start_month_day = pd.to_datetime(summer_start).strftime('%m-%d')
winter_start_month_day = pd.to_datetime(winter_start).strftime('%m-%d')

def is_summer(date):
        year = date.year
        summer_start_date = pd.to_datetime(f"{year}-{summer_start_month_day}")
        winter_start_date = pd.to_datetime(f"{year}-{winter_start_month_day}")
        return summer_start_date <= date < winter_start_date

df['is_summer'] = df['date'].apply(is_summer)

# Fill with a blank column for each callsign
for callsign in hems_rota.callsign:
    df[callsign] = 0

df

,date,is_summer,H70,CC70,H71,CC71,CC72
0,2025-02-14,False,0,0,0,0,0
1,2025-02-15,False,0,0,0,0,0
2,2025-02-16,False,0,0,0,0,0
3,2025-02-17,False,0,0,0,0,0
4,2025-02-18,False,0,0,0,0,0
...,...,...,...,...,...,...,...
361,2026-02-10,False,0,0,0,0,0
362,2026-02-11,False,0,0,0,0,0
363,2026-02-12,False,0,0,0,0,0
364,2026-02-13,False,0,0,0,0,0


In [80]:
def update_availability(df, rota):
    df = df.copy()

    for _, row in rota.iterrows():
        callsign = row['callsign']
        summer_start, winter_start = row['summer_start'], row['winter_start']
        summer_end, winter_end = row['summer_end'], row['winter_end']

        for i in range(len(df)):
            is_summer = df.at[i, 'is_summer']
            start_time = summer_start if is_summer else winter_start
            end_time = summer_end if is_summer else winter_end

            # Handle end time past midnight
            if end_time < start_time:
                duration = (24 - start_time) + end_time
                df.at[i, callsign] = duration

                if i + 1 < len(df):  # Add extra hours to next day
                    df.at[i + 1, callsign] = df.at[i + 1, callsign] + end_time
            else:
                df.at[i, callsign] = end_time - start_time

    # Adjust for simulation start time
    first_day = df.iloc[0]
    for callsign in rota['callsign']:
        if first_day[callsign] > 0:
            first_day[callsign] = max(0, first_day[callsign] - warm_up_end.hour)
    df.iloc[0] = first_day

    # Adjust for simulation end time
    last_day = df.iloc[-1]
    for callsign in rota['callsign']:
        if last_day[callsign] > 0:
            last_day[callsign] = min(last_day[callsign], sim_end.hour)
    df.iloc[-1] = last_day

    return df

update_availability(df, hems_rota)

C:\Users\Sammi\AppData\Local\Temp\ipykernel_67572\87864462.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Sammi\AppData\Local\Temp\ipykernel_67572\87864462.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,is_summer,H70,CC70,H71,CC71,CC72
0,2025-02-14,False,11,11,2,2,2
1,2025-02-15,False,19,19,10,10,10
2,2025-02-16,False,19,19,10,10,10
3,2025-02-17,False,19,19,10,10,10
4,2025-02-18,False,19,19,10,10,10
...,...,...,...,...,...,...,...
361,2026-02-10,False,19,19,10,10,10
362,2026-02-11,False,19,19,10,10,10
363,2026-02-12,False,19,19,10,10,10
364,2026-02-13,False,19,19,10,10,10


In [81]:
df_avail = update_availability(df, hems_rota)
df_avail

C:\Users\Sammi\AppData\Local\Temp\ipykernel_67572\87864462.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Sammi\AppData\Local\Temp\ipykernel_67572\87864462.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,is_summer,H70,CC70,H71,CC71,CC72
0,2025-02-14,False,11,11,2,2,2
1,2025-02-15,False,19,19,10,10,10
2,2025-02-16,False,19,19,10,10,10
3,2025-02-17,False,19,19,10,10,10
4,2025-02-18,False,19,19,10,10,10
...,...,...,...,...,...,...,...
361,2026-02-10,False,19,19,10,10,10
362,2026-02-11,False,19,19,10,10,10
363,2026-02-12,False,19,19,10,10,10
364,2026-02-13,False,19,19,10,10,10


In [82]:
avail_hours = df_avail.drop(columns=['is_summer']).sum(axis=0, numeric_only=True)

## Headline figure calculation

In [83]:
utilisation_df_overall

,callsign,vehicle_type,resource_use_duration,total_available_minutes_in_sim,callsign_group,perc_time_in_use,PRINT_perc
0,C70,car,62066.6,416100,70,0.149163,14.9%
1,C71,car,19790.8,219000,71,0.090369,9.0%
2,C72,car,24608.0,219000,72,0.112365,11.2%
3,H70,helicopter,36780.8,416100,70,0.088394,8.8%
4,H71,helicopter,11105.6,240960,71,0.046089,4.6%


In [84]:
vehicle_type = "helicopter"
utilisation_df_overall[utilisation_df_overall["vehicle_type"] == vehicle_type].mean(numeric_only=True)['perc_time_in_use']

0.06724155672431639

In [85]:
utilisation_df_overall.groupby('callsign_group').count()[['callsign']]

,callsign
callsign_group,
70,2
71,2
72,1


In [86]:
vehicles_per_callsign_group = utilisation_df_overall.groupby('callsign_group').count()[['callsign']]


car_only = vehicles_per_callsign_group[vehicles_per_callsign_group['callsign'] == 1]
car_only


,callsign
callsign_group,
72,1


In [87]:
utilisation_df_overall[utilisation_df_overall["callsign_group"].isin(car_only.reset_index().callsign_group.values)].mean(numeric_only=True)['perc_time_in_use']


0.11236529680365297

In [88]:
# backupcar_only = vehicles_per_callsign_group[vehicles_per_callsign_group['callsign'] == 2]
# utilisation_df_overall = utilisation_df_overall[utilisation_df_overall["vehicle_type"] == "car"]
# utilisation_df_overall[utilisation_df_overall["callsign_group"].isin(backupcar_only.reset_index().callsign_group.values)].mean(numeric_only=True)['perc_time_in_use']

In [89]:
_utilisation_result_calculation.make_SIMULATION_utilisation_headline_figure("helicopter", utilisation_df_overall)

0.06724155672431639

In [90]:
_utilisation_result_calculation.make_SIMULATION_utilisation_headline_figure("helicopter backup car", utilisation_df_overall)

0.11976582552271088

In [91]:
_utilisation_result_calculation.make_SIMULATION_utilisation_headline_figure("solo car", utilisation_df_overall)

0.11236529680365297